# Στοχαστικές Διαδικασίες - 2η Εργαστηριακή Άσκηση

## Στοιχεία Φοιτητή:
* Ονοματεπώνυμο: Ιωάννης Δάρας
* Αριθμός Μητρώου: 03115018
* Σχολή: ΗΜΜΥ, ΕΜΠ
* Εξάμηνο: 8ο
* E-mail: daras.giannhs@gmail.com

## Στόχος εργαστηρίου

Στο εργαστήριο αυτό, χρησιμοποιούμε τη μέθοδο Monte Carlo προκειμένου να εκτιμήσουμε πιθανότητες γεγονότων που αφορούν στοχαστικές διαδικασίες αλλά και να προσεγγίσουμε τη μέση τιμή και τη διασπορά στοχαστικών διαδικασιών.

### Ασκήσεις 33-35

Αρχικά, κάνουμε τα απαραίτητα imports.

In [1]:
import random
from simple_markov_chain_lib import markov_chain
import numpy as np

Στη συνεχεια, φτιάχνουμε τη ζητούμενη αλυσίδα.

In [2]:
p = 1/6
init_probs = {1: 1.0} 
markov_table = {
    1: {2: 1.},
    2: {2: 2/3, 3: 1/3},
    3: {1: p, 2: 1-p}
}
mc = markov_chain(markov_table, init_probs)

Θέλουμε να εκτιμήσουμε την πιθανότητα: 

$$P[X_{20} = 3]$$.


Για να είμαστε ακριβρείς, κάνουμε M εκτιμήσεις αυτής της πιθανότητας. Κάθε εκτίμηση γίνεται παρατηρώντας την κατάσταση της αλυσίδας N φορές.

Εκτός από την εκτίμηση της μέσης τιμής των εκτιμήσεων, ενδιαφερόμαστε και για την διασπορά τους. 
Όλα αυτά φαίνονται στο ακόλουθο κομμάτι κώδικα:

In [3]:
def get_avg_var_of_estimations(mc, M, N, steps, target_state):
    ''' 
    Estimate the E[A], V[A] where A = P[X_steps = target_state]
    Args:
        mc: markov chain
        M: number of estimations (int)
        N: number of samples (int)
        steps: the step we are interested in (int)
    Returns: 
        (a, b): the average estimation and the variance of the estimations
    '''
    estimations = np.empty(M)
    for i in range(M):
        counter = 0
        for j in range(N):
            mc.start()
            for k in range(steps):
                mc.move()
            if mc.running_state == target_state:
                counter += 1
        estimations[i] = counter / N
    return np.average(estimations), np.var(estimations)

Στη συνέχεια, φτιάχνουμε μια συνάρτηση για το θεωρητικό υπολογισμό της μέσης τιμής

In [4]:
def get_theoretical_avg(prob, init_state, finish_state):
    return np.linalg.matrix_power(prob, 200)[init_state - 1][finish_state - 1]

### Απαντήσεις ερωτημάτων

    1. 

In [5]:
E0, V0 = get_avg_var_of_estimations(mc, 50, 200, 20, 3)
E1, V1 = get_avg_var_of_estimations(mc, 50, 20000, 20, 3)
print('Average estimation N=200:{}, N=20000:{}'.format(E0, E1))

Average estimation N=200:0.24049999999999996, N=20000:0.24022099999999993


    2. 
 

In [6]:
prob = np.array([
        [0, 1, 0],
        [0, 2/3, 1/3],
        [p, 1-p, 0]])
print(get_theoretical_avg(prob, 1, 3))

0.2399999999999995


Όπως φαίνεται παραπάνω, τα αποτελέσματα συμφωνούν σε μεγάλο βαθμό με τη θεωρητική τιμή. Μάλιστα, όσο αυξάνεται ο αριθμός των δειγμάτων τόσο μειώνεται η διαφορά της θεωρητικής τιμής από την εκτιμώμενη.

    3.

In [7]:
print('Variance of estimations N=200:{}, N=20000:{}'.format(V0, V1))

Variance of estimations N=200:0.00131125, N=20000:1.3570909000000016e-05


    4.

In [8]:
init_probs = {3: 1.0}
mc = markov_chain(markov_table, init_probs)
E0, V0 = get_avg_var_of_estimations(mc, 50, 200, 20, 3)
E1, V1 = get_avg_var_of_estimations(mc, 50, 20000, 20, 3)
print('Average estimation N=200:{}, N=20000:{}'.format(E0, E1))
print('Variance of estimations N=200:{}, N=20000:{}'.format(V0, V1))

Average estimation N=200:0.24359999999999998, N=20000:0.239934
Variance of estimations N=200:0.0006860400000000002, N=20000:7.535543999999996e-06


## Άσκηση 40

Αρχικά, φτιάχνουμε την πίνακα της Μαρκοβιανής αλυσίδας.

In [9]:
p = 0.6
markov_table = {
    '0-0': {'15-0': p, '0-15': 1-p},
    '15-0': {'30-0': p, '15-15': 1-p},
    '15-15': {'30-15': p, '15-30': 1-p},
    '30-15': {'40-15': p, 'Deuce': 1-p},
    '30-0': {'40-0': p, '30-15': 1-p},
    '40-0': {'GameA': p, '40-15': 1-p},
    '40-15': {'GameA': p, 'AdvA': 1-p},
    'GameA': {'GameA': 1},
    'AdvA': {'GameA': p, 'Deuce': 1-p},
    'Deuce': {'AdvA': p, 'AdvB': 1-p},
    'AdvB': {'Deuce': p, 'GameB': 1-p},
    'GameB': {'GameB': 1},
    '0-15': {'15-15': p, '0-30': 1-p},
    '0-30': {'15-30': p, '0-40': 1-p},
    '15-30': {'Deuce': p, '15-40': 1-p},
    '0-40': {'15-40': p, 'GameB': 1-p},
    '15-40': {'AdvB': p, 'GameB': 1-p},
}
init_probs = {'0-0': 1.0}
mc = markov_chain(markov_table, init_probs)

Θέλουμε να εκτιμήσουμε την πιθανότητα νίκης με τη μέθοδο Monte Carlo. Γνωρίζουμε, ότι σε ένα παιχνίδι τένις δεν υπάρχει ισοπαλία, δηλαδή σε κάποια χρονική στιγμή θα κερδίσει κάποιος. Παρόλα αυτά, υπάρχει κύκλος στον οποίο μπορεί να μείνει για κάποιο χρόνο η αλυσίδα. Για παράδειγμα, μπορεί να συμβαίνει για πολύ χρόνο η κυκλική ακολουθία: AdvA, Deuce, AdvB. Επειδή υπάρχει θετική πιθανότητα να φύγουμε από αυτόν τον κύκλο, γνωρίζουμε ότι κάτι τέτοιο θα συμβεί, δηλαδή θα φτάσουμε σε μια από τις καταβόθρες GameA, GameB της στοχαστικής διαδικασίας. Όταν φτάσουμε σε μια τελική κατάσταση, κάνουμε την μέτρηση του νικητή και συνεχίζουμε.

Όλα αυτά, φαίνονται στο ακόλουθο κομμάτι κώδικα:

In [10]:
M = 50  # number of estimations
N = 2000  # number of samples
estimations = np.empty(M)
for i in range(M):
    counter = 0
    for j in range(N):
        mc.start()
        while mc.running_state not in ['GameA', 'GameB']:
            mc.move()
        if (mc.running_state == 'GameA'):
            counter += 1
    estimations[i] = counter / N

print(np.average(estimations))

0.73523
